In [1]:
'''
Messy reformation
'''
import json
import re

mydict = {}
mylist = []
mystr = ""
mybool = False

def messyReform(data):
    tlist = []
    if type(data) == type(mylist):
        for element in data:
            newdict = {}
            newdict['label'] = 'dummy'
            newdict['type'] = 'dummy'
            newdict['children'] = messyReform(element)
            tlist.append(newdict)
    else:
        for x in data.keys():
            if type(data[x]) != type(mylist) and type(data[x]) != type(mydict):
                newdict = {}
                newdict['label'] = data[x]
                newdict['type'] = x
                newdict['children'] = []
                tlist.append(newdict)
            else:
                newdict = {}
                newdict['label'] = x
                newdict['type'] = x
                newdict['children'] = messyReform(data[x])
                tlist.append(newdict)
    return tlist

In [8]:
'''
Main part
Messy reformation
'''

import os
import shutil

now = ''
nowtree=''
# flag=1
# if flag==1:
#     now='\\D\\Temp2\\'
#     nowtree='after_tree'
# else:
#     now='\\Before\\Temp2\\'
#     nowtree='before_tree'

base_path = 'D:\\Thesis\\Undergraduate-Thesis\\PHP\\All Codes\\Dataset\\Temp2\\'
#directories = [files for root, dirs, files in os.walk(base_path)]
directories = [x
               for x in os.walk(base_path)]
for x in directories[0][1]:
    filelist = [files for root, dirs, files in os.walk(base_path+x)]
    print(filelist)
    for y in filelist[0]:
        if 'Parsed' in y and '_Parsed' not in y:
            if(os.stat(base_path+x+"\\"+y).st_size == 0):
                print(x)
                shutil.rmtree(base_path+x)
                break
            with open(base_path+x+"\\"+y) as f:
                data = json.load(f)
            newdict = {}
            newdict['label'] = "root"
            newdict['type'] = "root"
            newdict['children'] = messyReform(data["sqlscript"]["statement"])
            with open(base_path+x+"\\_"+y, 'w') as f:
                json.dump(newdict, f)
                
    

[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeParsed1.json', 'beforeQuery1.sql', 'beforeXML1.xml']]
[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeParsed1.json', 'beforeQuery1.sql', 'beforeXML1.xml']]
[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeParsed1.json', 'beforeQuery1.sql', 'beforeXML1.xml']]
[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeQuery1.sql']]
[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeQuery1.sql']]
[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeQuery1.sql']]
[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeQuery1.sql']]
[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeQuery1.sql']]
[['afterParsed1.json', 'afterParsed2.json', 'afterQuery1.sql', 'afterQuery2.sql', 'afterXML1.xml', 'afterXML2.xml', 'beforeQuery1.sql', 'beforeQuery2.sql']]
[['afterParsed1.json', 'afterQuery1.sql', 'afterXML1.xml', 'beforeParsed1.json', 'b

In [9]:


'''
Travarce the JSON and Return path to the SQL Query String
children - a list
repeat - is multiple Query in one file then 
        repeat = 0 will find first ocurance, 
        repeat = 1 will find second ocurance
'''

mystr = ""

def find_string(children):
    path = []           # contains path to the query string
    # same shape as "path", continuous 1 if connected to ocurance of query, 0 otherwise
    marker = []

    if len(children) == 0:
        return False, path, marker

    for i in range(len(children)):
        # print(children[i])
        if 'label' not in children[i].keys():
            continue
        if type(children[i]['label']) == type(mystr) and re.search(r'DELETE.*FROM.*|SELECT.*FROM.*|INSERT.*INTO.*|UPDATE.*SET.*', children[i]['label'], re.IGNORECASE) != None:
            path.append(i)
            marker.append(1)
            return True, path, marker
        found, nextpath, nextmarker = find_string(children[i]['children'])
        if found == True:
            path.append(i)
            if children[i]['label'] == "PLUS" and nextmarker[0] == 1:
                marker.append(1)
            else:
                marker.append(0)
            path = path + nextpath
            marker = marker + nextmarker
            return True, path, marker

    return False, path, marker


'''
Travarce the children and replace "sqldata" in place using marker
'''


def travarce_replace(children, sqldata, path, marker, pos):
    if pos >= len(marker):
        return children
    if marker[pos] == 0:
        children[path[pos]]['children'] = travarce_replace(
            children[path[pos]]['children'], sqldata, path, marker, pos+1)
    else:
        children[path[pos]] = sqldata

    return children


In [14]:
'''
Main part of Replacing SQL parsed datat in Edit Tree
Change 'Before' 'After' to parse them
'''

import os
import json
import re

base_path = 'D:\\Thesis\\Undergraduate-Thesis\\PHP\\All Codes\\Dataset\\Temp2\\'
#directories = [files for root, dirs, files in os.walk(base_path)]
directories = [x
               for x in os.walk(base_path)]
for x in directories[0][1]:
    #fileList = []
    print(x)
    try:
        n = int(x)
        with open('D:\\Thesis\\Undergraduate-Thesis\\PHP\\All Codes\\Dataset\\'+ str(n) +"\\editTree.json") as f:
            data = json.load(f)
    except:
        continue
    
    children = data['before_tree']['children']
    filelist = [files for root, dirs, files in os.walk(base_path+now+x)]
    for y in filelist[0]:
        
        if '_beforeParsed' in y and os.stat(base_path+now+x+"\\"+y).st_size > 0:
            #fileList.append(y)
            print(x, y)
            with open(base_path+now+x+"\\"+y) as r:
                sqldata = json.load(r)
                
            _, path, marker = find_string(children)
            children = travarce_replace(children, sqldata, path, marker, 0)
            
            with open('D:\\Thesis\\Undergraduate-Thesis\\PHP\\All Codes\\Dataset\\'+ str(n) +"\\editTree.json", 'w') as outfile:
                json.dump(data, outfile)
    #print(n, x, fileList)
    
  
'''
i = 0
for x in directoryList:
    with open(x) as r:
        sqldata = json.load(r)
    _ , path, marker, _ = find_string(children, i++)
    children = find_replace(children, sqldata, path, marker, 0)
'''

1
1 _beforeParsed1.json
10
10 _beforeParsed1.json
2
2 _beforeParsed1.json
3
4
5
6
7
8
9
9 _beforeParsed1.json


'\ni = 0\nfor x in directoryList:\n    with open(x) as r:\n        sqldata = json.load(r)\n    _ , path, marker, _ = find_string(children, i++)\n    children = find_replace(children, sqldata, path, marker, 0)\n'